## Dependencies

In [28]:
import random

random.seed(42)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os

from PIL import Image
import imagesize

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras import layers, models

from scipy.ndimage import sobel
from keras import callbacks


In [2]:
bloodcells = os.listdir("bloodcells_dataset")

print(bloodcells)

bloodcells.remove('.DS_Store')
bloodcells.remove('All_Images')

['basophil', 'neutrophil', '.DS_Store', 'ig', 'monocyte', 'All_Images', 'eosinophil', 'erythroblast', 'lymphocyte', 'platelet']


In [3]:
images = os.listdir('bloodcells_dataset/' + bloodcells[0])
df = pd.DataFrame(data = {'images': images, 'type': bloodcells[0], 'height': np.nan, 'width': np.nan})

for i in range(1, len(bloodcells)):

    images = os.listdir('bloodcells_dataset/' + bloodcells[i]) # jpg string paths
    
    images_df = pd.DataFrame(data = {'images': images, 'type': bloodcells[i], 'height': np.nan, 'width': np.nan})
    
    df = pd.concat([df, images_df])

le = LabelEncoder()

df['width'] = df['images'].apply(lambda x: imagesize.get('bloodcells_dataset/All_Images/' + x)[0])
df['height'] = df['images'].apply(lambda x: imagesize.get('bloodcells_dataset/All_Images/' + x)[1])
df['type'] = le.fit_transform(df['type'])
df.index = range(len(df.index))

df = df[df['images'].str.contains('copy') == False]


df

,images,type,height,width
0,BA_689200.jpg,0,363,360
1,BA_883452.jpg,0,363,360
2,BA_382161.jpg,0,369,366
3,BA_175579.jpg,0,363,360
4,BA_775722.jpg,0,363,360
...,...,...,...,...
17087,PLATELET_495918.jpg,7,363,360
17088,PLATELET_897238.jpg,7,363,360
17089,PLATELET_750430.jpg,7,363,360
17090,PLATELET_810431.jpg,7,363,360


In [4]:
df[['height', 'width']].value_counts().reset_index(name = 'count')

,height,width,count
0,363,360,16639
1,369,366,250
2,360,360,198
3,361,360,2
4,360,359,1
5,360,361,1
6,360,362,1


In [5]:
category_counts = df[['type']].value_counts().reset_index(name = 'count')

total_num_images = category_counts['count'].sum()

category_counts

,type,count
0,6,3329
1,1,3117
2,3,2895
3,7,2348
4,2,1551
5,5,1420
6,0,1218
7,4,1214


In [6]:
resize_pixels = 96

In [7]:
# Weighted Sampling

# high representation - types 1, 3, 6, 7 (sample 1500)
# low representation - types 0, 2, 4, 5 (sample 1000)

df_weighted_sampling_train = df[df['type'] == 6].sample(1500)

for i in range(1, len(category_counts)):

    type = category_counts['type'][i]

    if category_counts['count'][i] >= 2000:
        add_samples = df[df['type'] == type].sample(1500)
    else: 
        add_samples = df[df['type'] == type].sample(1000)

    df_weighted_sampling_train = pd.concat([df_weighted_sampling_train, add_samples])

df_weighted_sampling_test = df[~df['images'].isin(df_weighted_sampling_train['images'])]

# Train
weighted_sampling_train = np.array(
    [np.array(Image.open('bloodcells_dataset/All_Images/' + image).resize((resize_pixels, resize_pixels))) for image in df_weighted_sampling_train['images']]
)


# Test
weighted_sampling_test = np.array(
    [np.array(Image.open('bloodcells_dataset/All_Images/' + image).resize((resize_pixels, resize_pixels))) for image in df_weighted_sampling_test['images']]
)


In [9]:
#apply Sobel filter for edge detection purposes

#class EdgeDetection + Blurring, greyscale whatever

def edge_detection(input_data):
    '''
    Args:
        input_data (array): image array
    Returns:
        x_sobel (array): horizontal sobel-filtered array with same dimension as input_data
        y_sobel (array): vertical sobel-filtered array with same dimension as input_data
        norm (array): combined array magnitude after filter application
    '''
    input_data = input_data.astype('int32')
    x_sobel = sobel(input_data, axis = 0) #compute horizontal gradient
    y_sobel = sobel(input_data, axis = 1) #compute vertical gradient
    norm = np.sqrt(x_sobel**2 + y_sobel**2)
    norm *= 255.0 / np.max(norm)  # normalization
    return x_sobel, y_sobel, norm



In [10]:
#apply sobel filter vertically and horizontally for edge detection
x_sobel, y_sobel, norm = edge_detection(weighted_sampling_train)
x_sobel_test, y_sobel_test, norm_test = edge_detection(weighted_sampling_test)

In [61]:
def plot_edges(image, x_sobel, y_sobel, norm):
    '''
    Plot images after edge detection.
    
    Args:
        input_data
        x_sobel
        y_sobel
        norm
    
    Returns 4 subplots revealing horizontal + vertical edge detection before and after.
    '''
    image = image.astype('int32')
    x_sobel = x_sobel.astype("int32")
    y_sobel = y_sobel.astype("int32")
    norm = norm.astype("int32")
    fig, ax = plt.subplots(2, 2, figsize=(100, 100))
    #plt.gray()  # show the filtered result in grayscale
    ax[0, 0].imshow(image[1])
    ax[0, 1].imshow(x_sobel[1])
    ax[1, 0].imshow(y_sobel[1])
    ax[1, 1].imshow(norm[1])
    plt.gray()  # show the filtered result in grayscale
    titles = ["original", "horizontal", "vertical", "norm"]
    for i, ax in enumerate(ax.ravel()):
        ax.set_title(titles[i])
        ax.axis("off")
    plt.show()

In [62]:
plot_edges(weighted_sampling_train, x_sobel, y_sobel, norm)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


#train normalization
weighted_sampling_train_images = tf.convert_to_tensor(weighted_sampling_train)

weighted_sampling_train_labels = tf.convert_to_tensor(df_weighted_sampling_train['type'])

# Test normalization
weighted_sampling_test_images = tf.convert_to_tensor(weighted_sampling_test_images)

weighted_sampling_test_labels = tf.convert_to_tensor(df_weighted_sampling_test['type'])


In [12]:
weighted_sampling_train_images = tf.convert_to_tensor(norm)

weighted_sampling_train_labels = tf.convert_to_tensor(df_weighted_sampling_train['type'])

# Test normalization
weighted_sampling_test_images = tf.convert_to_tensor(norm_test)

weighted_sampling_test_labels = tf.convert_to_tensor(df_weighted_sampling_test['type'])

In [13]:
# Proportional Sampling

num_samples = int(0.8 * total_num_images)

category_counts['prop'] = category_counts['count'] / total_num_images
category_counts['prop_samples'] = category_counts['prop'] * num_samples
category_counts['prop_samples'] = category_counts['prop_samples'].astype('int32')

df_prop_sampling_train = df[df['type'] == 6].sample(2663)

for i in range(1, len(category_counts)):

    type = category_counts['type'][i]

    samples = category_counts['prop_samples'][i]

    add_samples = df[df['type'] == type].sample(samples)

    df_prop_sampling_train = pd.concat([df_prop_sampling_train, add_samples])

df_prop_sampling_test = df[~df['images'].isin(df_prop_sampling_train['images'])]

# # Train
 weighted_prop_train_images = np.array(
    [np.array(Image.open('bloodcells_dataset/All_Images/' + image).resize((resize_pixels, resize_pixels))) for image in df_prop_sampling_train['images']]
)

weighted_prop_train_images = tf.convert_to_tensor(weighted_prop_train_images / 255.0)

weighted_prop_train_labels = tf.convert_to_tensor(df_prop_sampling_train['type'])

# Test
weighted_prop_test_images = np.array(
    [np.array(Image.open('bloodcells_dataset/All_Images/' + image).resize(((resize_pixels, resize_pixels))) for image in df_prop_sampling_test['images']]
)

weighted_prop_test_images = tf.convert_to_tensor(weighted_prop_test_images / 255.0)

weighted_prop_test_labels = tf.convert_to_tensor(df_prop_sampling_test['type'])

In [45]:


model = models.Sequential(
    
    [
        # 32 kernels, 3 by 3 kernel
        # input is 300 by 300 pixels with 3 color channels
        layers.Conv2D(64, (3, 3), activation = 'relu', input_shape = (resize_pixels, resize_pixels, 3)),
        
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(32, (3, 3), activation = 'relu'),
        
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), activation = 'relu'),

        # flatten into 1d array
        layers.Flatten(),

        # Neural network
        layers.Dense(64, activation = 'relu'),

        # 10 different categories
        layers.Dense(8) 
    ]
    
)

model.compile(optimizer = 'adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), # using from_logits = True b/c no softmax layer
              metrics = ['accuracy'])

earlystopping = callbacks.EarlyStopping(monitor="val_accuracy",
                                        mode="min",
                                        patience=5,
                                        restore_best_weights=True)

#history = model.fit(weighted_sampling_train_images, 
                    #weighted_sampling_train_labels, 
                    #epochs = 9, 
                    #batch_size = 64)

history = model.fit(weighted_sampling_train_images, 
                    weighted_sampling_train_labels, 
                    epochs = 5, 
                    batch_size = 32,
                    #validation_data=(weighted_sampling_test_images, weighted_sampling_test_labels),
                    #callbacks=[earlystopping]
                   )


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 63ms/step - accuracy: 0.6421 - loss: 4.8375
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 63ms/step - accuracy: 0.9315 - loss: 0.2052
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 62ms/step - accuracy: 0.9573 - loss: 0.1206
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 63ms/step - accuracy: 0.9754 - loss: 0.0690
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 71ms/step - accuracy: 0.9702 - loss: 0.0798


In [46]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
history_df.loc[:, ['accuracy', 'val_accuracy']].plot()
plt.show()

KeyError: "['val_loss'] not in index"

In [47]:
predictions = model.predict(weighted_sampling_test_images)

predictions = predictions.argmax(axis = 1)

222/222 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step


In [48]:
np.array(predictions == weighted_sampling_test_labels).sum() / len(predictions)

0.8526508742244783

In [ ]:
def plot_decision_region()

## After classifying, create "average" image of each class

 In medical image fusion, combining multiple images from single or multiple modalities reduces the redundancy and augments the usefulness and capabilities of medical images in diagnosis. While we are simply classifying bloodcells, we thought it would be interesting

## Edge Detection

In [78]:
model2 = models.Sequential(
    
    [
        # 32 kernels, 3 by 3 kernel
        # input is 300 by 300 pixels with 3 color channels
        layers.Conv2D(64, (3, 3), activation = 'relu', input_shape = (resize_pixels, resize_pixels, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation = 'relu'),
        layers.Conv2D(64, (3,3), activation = "relu"),
        layers.Dropout(0.2),
        layers.Conv2D(64, (3,3), activation = "relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3,3), activation = "relu"),
        layers.Conv2D(64, (3, 3), activation = 'relu'),
        layers.Dropout(0.2),
        layers.Conv2D(64, (3,3), activation = "relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3,3), activation = "relu"),
        layers.Conv2D(64, (3,3), activation = "relu"),
        # flatten into 1d array
        layers.Flatten(),

        # Neural network
        layers.Dense(64, activation = 'relu'),

        # 10 different categories
        layers.Dense(8) 
    ]
    
)

model2.compile(optimizer = 'adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), # using from_logits = True b/c no softmax layer
              metrics = ['accuracy'])

earlystopping = callbacks.EarlyStopping(monitor="val_accuracy",
                                        mode="min",
                                        patience=5,
                                        restore_best_weights=True)

#history = model.fit(weighted_sampling_train_images, 
                    #weighted_sampling_train_labels, 
                    #epochs = 9, 
                    #batch_size = 64)

history = model2.fit(weighted_sampling_train_images, 
                    weighted_sampling_train_labels, 
                    epochs = 10, 
                    batch_size = 64,
                    #validation_data=(weighted_sampling_test_images, weighted_sampling_test_labels),
                    #callbacks=[earlystopping]
                   )


Epoch 1/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 37s 233ms/step - accuracy: 0.4596 - loss: 1.8548
Epoch 2/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 42s 268ms/step - accuracy: 0.8361 - loss: 0.4654
Epoch 3/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 38s 240ms/step - accuracy: 0.8707 - loss: 0.3606
Epoch 4/10
107/157 ━━━━━━━━━━━━━━━━━━━━ 11s 238ms/step - accuracy: 0.9014 - loss: 0.2689

KeyboardInterrupt: 

In [77]:
predictions = model2.predict(weighted_sampling_test_images)

predictions = predictions.argmax(axis = 1)

np.array(predictions == weighted_sampling_test_labels).sum() / len(predictions)

222/222 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step


0.9107445008460237

In [79]:
model2.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_69 (Conv2D)              │ (None, 94, 94, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_36 (MaxPooling2D) │ (None, 47, 47, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_70 (Conv2D)              │ (None, 45, 45, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_71 (Conv2D)              │ (None, 43, 43, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_72 (Conv2D)              │ (None, 41, 41, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_37 (MaxPooling2D) │ (None, 20, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_73 (Conv2D)              │ (None, 18, 18, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_74 (Conv2D)              │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_75 (Conv2D)              │ (None, 14, 14, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_38 (MaxPooling2D) │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 64)             │       200,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,163,258 (4.44 MB)

 Trainable params: 387,752 (1.48 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 775,506 (2.96 MB)